In [4]:
w = 2.0 # 
h = 1.0 
offset = 0.1
racing_setup = { 0: [[0, 1, h/2+offset], [-w/2, 1, h+offset], [w/2, 1, h+offset], [-w/2, 1, offset], [w/2, 1, offset]],
                            1: [[0.5, 3, h/2+offset], [-w/2+0.5, 3, h+offset], [w/2+0.5, 3, h+offset], [-w/2+0.5, 3, offset], [w/2+0.5, 3, offset]],
                            2: [[-0.5, 5, h/2+offset], [-w/2-0.5, 5, h+offset], [w/2-0.5, 5, h+offset], [-w/2-0.5, 5, offset], [w/2-0.5, 5, offset]],
                            3: [[0.5, 7, h/2+offset], [-w/2+0.5, 7, h+offset], [w/2+0.5, 7, h+offset], [-w/2+0.5, 7, offset], [w/2+0.5, 7, offset]],
                            4: [[1.0, 9, h/2+offset], [-w/2+1.0, 9, h+offset], [w/2+1.0, 9, h+offset], [-w/2+1.0, 9, offset], [w/2+1.0, 9, offset]]
                            }
passing_flag = [False for _ in range(len(racing_setup.keys()))]


In [5]:
passing_flag

[False, False, False, False, False]

In [11]:
for i, key in enumerate(racing_setup.keys()):
    print(racing_setup[key][1][1])
    # if passing_flag[i]:
    #     continue
    # if racing_setup[key][1][0] < pos[0] < racing_setup[key][2][0] and \
    #     racing_setup[key][1][1] - 0.5 < pos[1] < racing_setup[key][2][1] + 0.5 and \
    #     racing_setup[key][4][2] < pos[2] < racing_setup[key][1][2]:
    #     passing_flag[i] = True
    #     passing = True
    #     break

1
3
5
7
9


In [1]:
import os
import time
from datetime import datetime
import argparse
import gymnasium as gym
import numpy as np
import torch
import csv
from TD3.td3 import TD3Agent
from TD3.td3 import Actor, Critic,  ReplayBuffer
import numpy as np

from gym_pybullet_drones.utils.Logger import Logger
from gym_pybullet_drones.envs.HoverAviary import HoverAviary
from gym_pybullet_drones.envs.FlyThruGateAvitary import FlyThruGateAvitary
from gym_pybullet_drones.utils.utils import sync, str2bool
from gym_pybullet_drones.utils.enums import ObservationType, ActionType
from collections import deque

/home/tuan/Desktop/drone_rl_control/gym_pybullet_drones/envs/BaseAviary.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
pybullet build time: Sep 29 2025 13:26:22


In [2]:
DEFAULT_GUI = True
DEFAULT_RECORD_VIDEO = False
DEFAULT_OUTPUT_FOLDER = 'log_dir/td3_thrugate'
DEFAULT_COLAB = False

DEFAULT_OBS = ObservationType('kin') # 'kin' or 'rgb' , KIN - (1,12) , RGB - (3,64,64)
DEFAULT_ACT = ActionType('rpm') # 'rpm' or 'pid' or 'vel' or 'one_d_rpm' or 'one_d_pid'

# Tạo timestamp cho run này
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"training_data_{timestamp}.csv"

env = FlyThruGateAvitary(obs=DEFAULT_OBS, act=DEFAULT_ACT)

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000


/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [6]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
print(f"State dim: {state_dim}, Action dim: {action_dim}")

State dim: 1, Action dim: 1


In [8]:
import gymnasium as gym
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="rgb_array")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
print(f"State dim: {state_dim}, Action dim: {action_dim}")

State dim: 24, Action dim: 4


## Test envs for Racing condition 

In [6]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from gymnasium.vector import SyncVectorEnv

from SAC.sac_agent import SACAgent, ReplayBuffer, device
from gym_pybullet_drones.envs import SemiCircleRacingAviary
from gym_pybullet_drones.utils.enums import ObservationType, ActionType



def make_env(seed: int, spawn_mode: str = "random_track"):
    rng = np.random.default_rng(seed)

    def _init():
        env = SemiCircleRacingAviary(
            gui=False,
            record=False,
            obs=ObservationType.KIN,
            act=ActionType.RPM,
        )
        env._rng = rng
        env._spawn_mode = spawn_mode
        return env

    return _init

def reset_envs(vec_env, spawn_mode: str):
    options = {"spawn": spawn_mode}
    return vec_env.reset(options=options)

num_envs = 4
vec_env = SyncVectorEnv([make_env(i) for i in range(num_envs)])
obs, info = reset_envs(vec_env, "random_track")

# Check dimensions of state and action spaces
vec_env.single_action_space.shape[1]
obs.shape[1] if obs.ndim == 2 else obs.reshape(num_envs, -1).shape[1]

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3

/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


36

36

In [5]:
import os
import time
from datetime import datetime
import argparse
import numpy as np
import torch
import matplotlib.pyplot as plt
from PPO.ppo_agent import PPO

from gymnasium.vector import SyncVectorEnv
from gym_pybullet_drones.envs import SemiCircleRacingAviary
from gym_pybullet_drones.utils.utils import sync, str2bool
from gym_pybullet_drones.utils.enums import ObservationType, ActionType

def make_env(seed: int, gui: bool = False, record: bool = False):
    """Factory for vectorized environments with per-env seeding."""
    def _init():
        env = SemiCircleRacingAviary(
            gui=gui,
            record=record,
            obs=ObservationType.KIN,
            act=ActionType.RPM,
        )
        env._rng = np.random.default_rng(seed)
        return env
    return _init



DEFAULT_GUI = False
DEFAULT_RECORD_VIDEO = False
DEFAULT_OUTPUT_FOLDER = 'results'
DEFAULT_COLAB = False

num_envs = 8  # parallel envs to spawn on varied gates/tracks
reset_options = {
    "spawn": "random_track",           # sample start at different gates
    "random_track_layout": True,       # randomize semicircle radius/center/yaw
    "radius_range": (3.0, 6.0),
    "center_jitter": 1.0,
    "yaw_jitter": 0.4,
}

vec_env = SyncVectorEnv([make_env(seed=i, gui=DEFAULT_GUI, record=DEFAULT_RECORD_VIDEO) for i in range(num_envs)])
obs, info = vec_env.reset(options=reset_options)
obs.shape

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3

/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


(8, 36)

In [ ]:
shape